## 1. Graph Attention Networks

In [3]:
import torch
torchversion = torch.__version__

# Install PyTorch Scatter, PyTorch Sparse, and PyTorch Geometric
!pip install torch_geometric
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-{torchversion}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-{torchversion}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Numpy for matrices
import numpy as np
np.random.seed(0)

# Visualization
import networkx as nx
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

### Dataset

In [4]:
from torch_geometric.datasets import Planetoid,KarateClub

# Import dataset from PyTorch Geometric
dataset = Planetoid(root=".", name="cora")

data = dataset[0]


C:\Users\user\anaconda3\envs\test\lib\site-packages\torch_geometric\typing.py:60: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: Could not find module 'C:\Users\user\anaconda3\envs\test\Lib\site-packages\torch_scatter\_scatter_cuda.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
C:\Users\user\anaconda3\envs\test\lib\site-packages\torch_geometric\typing.py:98: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: Could not find module 'C:\Users\user\anaconda3\envs\test\Lib\site-packages\torch_sparse\_convert_cuda.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


### Print information about the dataset

In [12]:
# Print information about the dataset
print(f'Dataset: {dataset}')
print('-------------------')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of nodes: {data.x.shape[0]}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

# Print information about the graph
print(f'\nGraph:')
print('------')
print(f'Edges are directed: {data.is_directed()}')
print(f'Graph has isolated nodes: {data.has_isolated_nodes()}')
print(f'Graph has loops: {data.has_self_loops()}')

Dataset: cora()
-------------------
Number of graphs: 1
Number of nodes: 2708
Number of features: 1433
Number of classes: 7

Graph:
------
Edges are directed: False
Graph has isolated nodes: False
Graph has loops: False


In [13]:
from torch_geometric.utils import remove_isolated_nodes

isolated = (remove_isolated_nodes(data['edge_index'])[2] == False).sum(dim=0).item()
print(f'Number of isolated nodes = {isolated}')

Number of isolated nodes = 0


### Implement GAT

#### GAT model

In [18]:
import torch.nn.functional as F
import torch
from torch.nn import Linear, Dropout
from torch_geometric.nn import GCNConv, GATv2Conv
class GAT(torch.nn.Module):
  """Graph Attention Network"""
  def __init__(self, dim_in, dim_h, dim_out, heads=8):
    super().__init__()
    self.gat1 = GATv2Conv(dim_in, dim_h, heads=heads)
    self.gat2 = GATv2Conv(dim_h*heads, dim_out, heads=1)
    self.optimizer = torch.optim.Adam(self.parameters(),
                                      lr=0.005,
                                      weight_decay=5e-4)

  def forward(self, x, edge_index):
    h = F.dropout(x, p=0.6, training=self.training)
    h = self.gat1(x, edge_index)
    h = F.elu(h)
    h = F.dropout(h, p=0.6, training=self.training)
    h = self.gat2(h, edge_index)
    return h, F.log_softmax(h, dim=1)

### Train function

In [19]:
def accuracy(pred_y, y):
    """Calculate accuracy."""
    return ((pred_y == y).sum() / len(y)).item()

def train(model, data):
    """Train a GNN model and return the trained model."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = model.optimizer
    epochs = 5

    model.train()
    for epoch in range(epochs+1):
        # Training
        optimizer.zero_grad()
        _, out = model(data.x, data.edge_index)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        acc = accuracy(out[data.train_mask].argmax(dim=1), data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        # Validation
        val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
        val_acc = accuracy(out[data.val_mask].argmax(dim=1), data.y[data.val_mask])

        # Print metrics every 10 epochs
        if(epoch % 1 == 0):
            print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc: '
                  f'{acc*100:>6.2f}% | Val Loss: {val_loss:.2f} | '
                  f'Val Acc: {val_acc*100:.2f}%')

    return model

In [20]:
def test(model, data):
    """Evaluate the model on test set and print the accuracy score."""
    model.eval()
    _, out = model(data.x, data.edge_index)
    acc = accuracy(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])
    return acc

### Train GAT

In [21]:
%%time

# Create GAT model
gat = GAT(dataset.num_features, 8, dataset.num_classes)
print(gat)

GAT(
  (gat1): GATv2Conv(1433, 8, heads=8)
  (gat2): GATv2Conv(64, 7, heads=1)
)
CPU times: total: 0 ns
Wall time: 15.4 ms


In [22]:


# Train
train(gat, data)

# Test
acc = test(gat, data)
print(f'\nGAT test accuracy: {acc*100:.2f}%\n')

Epoch   0 | Train Loss: 1.934 | Train Acc:  19.29% | Val Loss: 1.94 | Val Acc: 17.40%
Epoch   1 | Train Loss: 1.702 | Train Acc:  80.71% | Val Loss: 1.83 | Val Acc: 53.40%
Epoch   2 | Train Loss: 1.486 | Train Acc:  91.43% | Val Loss: 1.69 | Val Acc: 66.00%
Epoch   3 | Train Loss: 1.283 | Train Acc:  96.43% | Val Loss: 1.59 | Val Acc: 69.60%
Epoch   4 | Train Loss: 1.078 | Train Acc:  97.14% | Val Loss: 1.47 | Val Acc: 72.40%
Epoch   5 | Train Loss: 0.924 | Train Acc:  95.71% | Val Loss: 1.36 | Val Acc: 72.80%

GAT test accuracy: 76.10%



### GATv2

In [23]:
class GATv2(torch.nn.Module):
    def __init__(self,num_features,dims, num_classes,  drop=0.0):
        super(GATv2, self).__init__()
        heads = 8
        self.h = None
        self.conv1 = GATv2Conv(num_features,dims, heads=heads, dropout = 0.3, concat=False)
        self.conv2 = GATv2Conv(dims, num_classes, heads=heads, concat=False, dropout=0.3)
        self.drop = torch.nn.Dropout(p=drop)
        self.optimizer = torch.optim.Adam(self.parameters(),lr=0.005,weight_decay=5e-4)
    def forward(self, x, edge_index,):
        x = F.elu(self.conv1(x, edge_index))
        x = self.drop(x)
        x = self.conv2(x, edge_index)
        return x, F.log_softmax(x, dim=1)

In [24]:
%%time

# Create GAT model
gat2 = GATv2(dataset.num_features, 8, dataset.num_classes)
print(gat2)

GATv2(
  (conv1): GATv2Conv(1433, 8, heads=8)
  (conv2): GATv2Conv(8, 7, heads=8)
  (drop): Dropout(p=0.0, inplace=False)
)
CPU times: total: 125 ms
Wall time: 2 ms


In [25]:
# Train
train(gat2, data)

# Test
acc = test(gat2, data)
print(f'\nGAT test accuracy: {acc*100:.2f}%\n')

Epoch   0 | Train Loss: 1.948 | Train Acc:  12.86% | Val Loss: 1.97 | Val Acc: 7.20%
Epoch   1 | Train Loss: 1.934 | Train Acc:  22.86% | Val Loss: 1.97 | Val Acc: 10.20%
Epoch   2 | Train Loss: 1.921 | Train Acc:  27.86% | Val Loss: 1.96 | Val Acc: 11.60%
Epoch   3 | Train Loss: 1.910 | Train Acc:  34.29% | Val Loss: 1.95 | Val Acc: 17.00%
Epoch   4 | Train Loss: 1.896 | Train Acc:  42.86% | Val Loss: 1.93 | Val Acc: 23.80%
Epoch   5 | Train Loss: 1.882 | Train Acc:  56.43% | Val Loss: 1.93 | Val Acc: 30.00%

GAT test accuracy: 55.80%



### RGAT (Relational GAT on AIFB graph)

In [27]:
!pip install rdflib
import os.path as osp
import time

import torch
import torch.nn.functional as F

from torch_geometric.datasets import Entities
from torch_geometric.nn import RGATConv

#path = osp.join(".", '..', 'data', 'Entities')'AIFB'
dataset = Entities(".", 'AIFB')
data = dataset[0]
data.x = torch.randn(data.num_nodes, 16)


In [30]:

class RGAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels,
                 num_relations):
        super().__init__()
        self.conv1 = RGATConv(in_channels, hidden_channels, num_relations)
        self.conv2 = RGATConv(hidden_channels, hidden_channels, num_relations)
        self.lin = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, x, edge_index, edge_type):
        x = self.conv1(x, edge_index, edge_type).relu()
        x = self.conv2(x, edge_index, edge_type).relu()
        x = self.lin(x)
        return F.log_softmax(x, dim=-1)

In [31]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)
model = RGAT(16, 16, dataset.num_classes, dataset.num_relations).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0005)



In [32]:

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index, data.edge_type)
    loss = F.nll_loss(out[data.train_idx], data.train_y)
    loss.backward()
    optimizer.step()
    return float(loss)


@torch.no_grad()
def test():
    model.eval()
    pred = model(data.x, data.edge_index, data.edge_type).argmax(dim=-1)
    train_acc = float((pred[data.train_idx] == data.train_y).float().mean())
    test_acc = float((pred[data.test_idx] == data.test_y).float().mean())
    return train_acc, test_acc



In [33]:

times = []
for epoch in range(1, 5):
    start = time.time()
    loss = train()
    train_acc, test_acc = test()
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Train: {train_acc:.4f} '
          f'Test: {test_acc:.4f}')
    times.append(time.time() - start)
print(f"Median time per epoch: {torch.tensor(times).median():.4f}s")

Epoch: 01, Loss: 1.4056, Train: 0.4071 Test: 0.3611
Epoch: 02, Loss: 1.3517, Train: 0.6571 Test: 0.6389
Epoch: 03, Loss: 1.2835, Train: 0.6357 Test: 0.6389
Epoch: 04, Loss: 1.1938, Train: 0.6143 Test: 0.6389
Median time per epoch: 0.2080s
